# Importar librerias

In [34]:
import numpy as np
import pandas as pd
import os

In [35]:
def changeHeaderDf(df):
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    return df

# Filtros

In [36]:
from os import walk
ruta_origen = 'kitsPowerTrain'

checker = 0
total = 0
# Se descartan CI MO MT SU por no ser relevantes
jobDict = {'CO':[],'DI':[],'MA':[],'MF':[],'MO':[],'TR':[]}
jobList = jobDict.keys()
columnIdentifier = ['Part No','PartNo','Part No..000','Part No.']

# Selector de Numero de Partes

In [37]:
# Read files and obtain values
for (dirpath, dirnames, filenames) in walk(ruta_origen):
    for file in filenames:
        if file.lower().endswith('.xls') or file.lower().endswith('.xlsx'):
            if file.startswith(tuple(jobDict.keys())):
                PNList=[]
                total +=1
                # Read Excel
                filePath = os.path.join(dirpath,file)
                df = pd.read_excel(filePath)
                for i in range(5):
                    if not PNList:
                        for label in columnIdentifier:
                            try:
                                # Getting list of Part No
                                PNList = df[label].tolist()
                                break
                            except:
                                PNList = []
                    else:
                        break
                    try:
                        df = changeHeaderDf(df)
                    except:
                        print(file)
                        break
                if PNList:
                    checker += 1
                strippedPNList = [str(s).strip() for s in PNList]
                # Appending to corresponding Job Control
                jobDict[file[0:2]].extend(strippedPNList)
        elif 'KIT MO' in dirpath and '$' not in file:
            PNList=[]
            total +=1
            # Read Excel
            filePath = os.path.join(dirpath,file)
            if file.lower().endswith('.xls') or file.lower().endswith('.xlsx'):
                df = pd.read_excel(filePath)
            elif file.lower().endswith('.csv'):
                df = pd.read_csv(filePath,sep=';')

            for i in range(5):
                if not PNList:
                    for label in columnIdentifier:
                        try:                          
                            PNList = df[label].tolist() # Getting list of Part No                          
                            break
                        except:
                            PNList = []
                else:
                    break
                try:
                    df = changeHeaderDf(df)
                except:
                    print(file)
                    break
            if PNList:
                checker += 1
            strippedPNList = [str(s).strip().replace('-','') for s in PNList]
            # Appending to corresponding Job Control
            jobDict['MO'].extend(strippedPNList)
print('Performance: ' + str(checker) +  "/" + str(total))

MO - Todos.xlsx
Performance: 247/273


# Exportar datos

In [38]:
import pickle
# Export Dictionary
with open('../03_Preprocesamiento/Datos/kitsComponente.pkl', 'wb') as f:
    pickle.dump(jobDict, f)